<a href="https://colab.research.google.com/github/errorcodecritical/JupyterML/blob/main/Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Room Occupancy Estimation using Machine Learning

## Introduction

This notebook aims to analyze and develop machine learning models for **Room Occupancy Estimation** using non-intrusive environmental sensors such as **temperature, light, sound, CO2, and PIR (Passive Infrared)**. The goal is to predict the number of occupants in a room based on sensor data.

This project is developed as part of the **Machine Learning Challenge (Aprendizagem Computacional), 2025** at **DEI, FCT, University of Coimbra**. The objective of the challenge is to identify a real-world classification problem, apply simple machine learning models, and evaluate their effectiveness.

## Dataset Information

The dataset used in this project is [**Room Occupancy Estimation**](https://archive.ics.uci.edu/dataset/864/room+occupancy+estimation), which consists of **10129 instances and 18 features**. The data was collected over four days in a controlled environment, where occupancy varied between **0 and 3 people**.

- **Sensor Types Used:**
  - **Temperature**
  - **Light**
  - **Sound**
  - **CO2**
  - **Passive Infrared (PIR)**

- **Experimental Setup:**
  - Data was collected in a **6m x 4.6m room**.
  - **7 sensor nodes** were deployed, transmitting data every **30 seconds**.
  - The PIR, CO2, and sound sensors required manual calibration.
  - The ground truth was manually recorded.

## Challenge Goals

The primary goals of this challenge are:
1. **Problem Identification:** Understand how environmental sensor data can be used for room occupancy estimation.
2. **Data Analysis:** Explore the dataset, clean, and preprocess it.
3. **Model Construction:** Implement at least **two simple machine learning models** (e.g., Decision Trees, K-Nearest Neighbors, Logistic Regression).
4. **Evaluation Metrics:** Assess model performance using appropriate evaluation metrics.
5. **Documentation & Submission:** Record findings, methodologies, and challenges faced.

## Next Steps

1. Load and preprocess the dataset.
2. Perform exploratory data analysis (EDA).
3. Train and evaluate machine learning models.
4. Compare model performance using visualization tools.

---

Install Required Libs: pip install scikit-learn pandas numpy scipy matplotlib seaborn

For report:
- DETERMINE PROBLEM, SOLUTION, OBJECTIVE;
- BREAKDOWN OF THE DATASET, DESCRIPTION OF VARIABLES AND TARGETS, NUMBER OF NULL VALUES 

In [ ]:
import pandas as pd
from ucimlrepo import fetch_ucirepo

#Converts hours, minutes, seconds to seconds for easier handling

def hms_to_seconds(t):
    h, m, s = [int(i) for i in t.split(':')]
    return 3600*h + 60*m + s
  
# fetch dataset 
room_occupancy_estimation = fetch_ucirepo(id=864) 
  
# data (as pandas dataframes) 
X = room_occupancy_estimation.data.features
y = room_occupancy_estimation.data.targets

# Get dataset information


# Get number of target elements of the same category
target_distribution = y.pivot_table(index=["Room_Occupancy_Count"], aggfunc="size")

# We can see the dataset is unbalanced. Ruh-roh!
X.iloc[:,1]

time_in_seconds = X.iloc[:, 1]
#room_occupancy_estimation(room_occupancy_estimation.loc["Date"])




0        10:49:41
1        10:50:12
2        10:50:42
3        10:51:13
4        10:51:44
           ...   
10124    08:58:07
10125    08:58:37
10126    08:59:08
10127    08:59:39
10128    09:00:09
Name: Time, Length: 10129, dtype: object

In [56]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set a theme for Seaborn plots
sns.set_theme()

# Histogram of "Room_Occupancy_Count"
plt.figure(figsize=(10, 6))
sns.barplot(x=["0", "1", "2", "3"], y=)
plt.title("Room Occupancy Distribution")
plt.show()

SyntaxError: expected argument value expression (1294885083.py, line 9)